In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
# Load test features and labels
X_test = np.load('rolling_window_sequences.npy')  # Replace with actual file path
metadata_test = pd.read_csv("sequence_metadata_with_RUL.csv")  # Replace with actual file path
y_test = metadata_test["RUL"].values
print("Test feature shape:", X_test.shape)
print("Test target shape:", y_test.shape)

Test feature shape: (17631, 30, 66)
Test target shape: (17631,)


In [3]:
# Split the data into train and validation sets (assuming no separate train set given)
X_train, X_val, y_train, y_val = train_test_split(X_test, y_test, test_size=0.3, random_state=42)

In [4]:
# Define GRU model creation function
def create_gru_model(input_shape, units=64, learning_rate=0.001, dropout_rate=0.2):
    model = Sequential()
    model.add(GRU(units, input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    return model

In [7]:
from keras.models import Sequential
from keras.layers import GRU, Dense, Dropout, Input
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

# Define GRU model creation function
def create_gru_model(input_shape, units=64, learning_rate=0.001, dropout_rate=0.2):
    model = Sequential()
    model.add(Input(shape=input_shape))   # modern way
    model.add(GRU(units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    return model

# Create the model
model = create_gru_model(
    input_shape=(X_train.shape[1], X_train.shape[2]),
    units=64,
    learning_rate=0.001,
    dropout_rate=0.2
)

# Callbacks for early stopping
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
]


In [8]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 13s 51ms/step - loss: 6934.3042 - mae: 71.7696 - val_loss: 5987.0142 - val_mae: 65.7349
Epoch 2/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 8s 43ms/step - loss: 5363.7539 - mae: 61.3603 - val_loss: 4748.0059 - val_mae: 56.9386
Epoch 3/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 8s 40ms/step - loss: 4263.5415 - mae: 53.4604 - val_loss: 3775.3596 - val_mae: 49.7418
Epoch 4/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 8s 40ms/step - loss: 3392.5078 - mae: 46.8816 - val_loss: 3000.3518 - val_mae: 43.6147
Epoch 5/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - loss: 2702.0828 - mae: 41.3382 - val_loss: 2362.5320 - val_mae: 38.3261
Epoch 6/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 8s 40ms/step - loss: 2106.9851 - mae: 36.2023 - val_loss: 1829.0902 - val_mae: 33.4662
Epoch 7/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step - loss: 1645.3313 - mae: 31.8985 - val_loss: 1439.9423 - val_mae: 29.6590
Epoch 8/30
193/193 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - loss: 1296.5588 - mae: 28.2741 - val_loss: 1134.5494 - 

In [9]:
# Evaluate on validation set
val_loss, val_mae = model.evaluate(X_val, y_val, verbose=0)
print(f'Validation MSE: {val_loss:.4f}, MAE: {val_mae:.4f}')

# Predict on test set
y_pred = model.predict(X_test).flatten()

# Calculate test MSE
test_mse = mean_squared_error(y_test, y_pred)
print(f'Test MSE: {test_mse:.4f}')

Validation MSE: 109.9515, MAE: 7.7195
551/551 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step
Test MSE: 95.9272
